# Hiena Multipass Parser

    hiena_mp()

A Hiena parser takes a Grammar and a Target, and generates a Dictionary tree.

    def hiena_mp(grammar, target, rulename) -> dict:
        ...

`hiena_mp()` is a multi-pass recursive-descent implementation suitable for small files with limited depth.

## Command Interpreter Mode

The 'hiena' parser can be specified as a command interpreter by putting a '#!' on the first line of the grammar file -and/or- in case the grammar is cached, a '#!' item in the grammar dict.

    { "#!": [hienapath, opts], 
       ...


In [45]:
import re
from Dcel import Dcel

class HienaStr(str):
    def __create__(self,string,rematchobj,match_index):
        super.__create__(string)
        
    def __init__(self,string,rematchobj,match_index):
        self.hiena_data_start = rematchobj.start(match_index)
        self.hiena_data_end = rematchobj.end(match_index)
        super.__init__(string)

def hiena_mp(g:dict, text:Dcel, rulename="$__start__"):
    
    assert(type(text) is Dcel)
    
    if (type(g) is Dcel
        and type(g.value) is dict):
        g = g.value
    
    # This begins its life as a list()
    # it collects the matches for a repeating grammar rule.
    
    tree = list()
    
    # Parse a layer of `text` using current `rulename` from grammar `g`.
    
    if rulename in g:
        
        # Hook for beginning of parsing a grammar.
        # The function is recursive, so the any rule could
        # be a start rule if the function is
        # called programmatically. When APath uses HienaMP
        # as a executable interpreter, it expects $__start__. 
        
        if rulename=="$__start__":
            rulename=g["$__start__"]
        
        # If $__start__ was not specified, this is the rulename
        # called in the function args. Otherwise, it is the rulename
        # resolved from $__start__.
        
        rule = g[rulename]
        
        # all matches within `text`.
        
        m = re.finditer(rule[0], 
                        str(text),
                        re.M
                       )
        
        # next rule that parses each match in `m`.
        nextrulename = rule[1]
        
        # branch rule
        if nextrulename != "":
            for ea in m:
                
                # create fragment Dcel from `text`
                map_fragment = text[ea.start(0):ea.end(0)]
                
                # parse match and collect result in list
                tree.append(hiena_mp(
                      g,
                      # ea.group(0),    # old string version
                      map_fragment,     # new Dcel fragment version
                      nextrulename
                     ))
        
        # terminal rule
        else:
            for ea in m:
                # terminal_value = ea.group(0)  # old string version
                map_fragment = text[ea.start(0):ea.end(0)]  # new Dcel fragment version
                
                # WIP: need to attach data_map to terminal_value object
                # ie. HienaValue(ea.group(valno),ea.start(valno),ea.end(valno))
                # ie. terminal_value.data_map = data_map
                
                # tree.append(terminal_value) 
                tree.append(map_fragment)
                
        # After all matches have been recursively parsed
        # create a dictionary keyed by `labels` provided in field 2
        # of the grammar rule.
        
        # If the `labels` are a dictionary {key:number,value:number}
        # then, extract the label from the text of the match.
        
        # FIXME: validate presence of 'key' and 'value' before entering this block.
        labels = rule[2]
        if type(labels) == dict:
            keyno = labels['key']
            valno = labels['value']
            # FIXME: eleminate this double-run of the grammar
            # by caching the results earlier in the function.
            m = re.finditer(rule[0], 
                    str(text),
                    re.M
                   )
            
            # HACK to populate empty Key-val-pairs with something useful.
            # This should propogate back to the underlier correctly.
            if(ea.start(valno) == -1):
                valno = keyno
            # end HACK
            tree = { ea.group(keyno):text[ea.start(valno):ea.end(valno)]
                    # WIP: need to attach data_map to terminal_value object
                    # ie. HienaValue(ea.group(valno),ea.start(valno),ea.end(valno))
                    for ea in m
                   }
            # WIP: need to attach a data_map to the tree
            # ie. tree.data_map
            return tree
        else:        
            tree = { k:v for k,v 
                    in zip(labels, 
                           tree
                          )}
        return tree

In [46]:
import json

LINE = '^.+$'
WORD = '[^ ]+'
CHAR = '\w'
entryschema = [str(i) for i in range(1,10)]
fieldschema = [
    'spec', 'file', 'vfstype', 
    'mntopts', 'freq', 'passno'
]
KVP = '([^= ,]+)(?:[=]([^=,]+))?'
kvpschema = {'key':1,'value':2}
fstabg = {
    "$__start__": "entry",
    "entry": [LINE, "field", entryschema ],
    "field": [WORD, "keyvaluepair", fieldschema],
    "keyvaluepair": [KVP, "", kvpschema]
}

sample = Dcel("""
one=1,two=2 three four
five six seven eight 23 4
""")

x = hiena_mp(fstabg,sample)
print(type(x))
print(x)

<class 'dict'>
{'1': {'spec': {'one': <Dcel.Dcel object at 0x7f529251abf0>, 'two': <Dcel.Dcel object at 0x7f529251aa70>}, 'file': {'three': <Dcel.Dcel object at 0x7f5292519fc0>}, 'vfstype': {'four': <Dcel.Dcel object at 0x7f5292518e80>}}, '2': {'spec': {'five': <Dcel.Dcel object at 0x7f5292519570>}, 'file': {'six': <Dcel.Dcel object at 0x7f5292518fd0>}, 'vfstype': {'seven': <Dcel.Dcel object at 0x7f529251bc70>}, 'mntopts': {'eight': <Dcel.Dcel object at 0x7f52925194b0>}, 'freq': {'23': <Dcel.Dcel object at 0x7f52925194e0>}, 'passno': {'4': <Dcel.Dcel object at 0x7f529251bd00>}}}


In [47]:
print(x['1']['spec']['one'].inspect())
print(sample.inspect())

address: <class 'slice'>:slice(4, 5, None)
        abspath: <class 'slice'>:slice(4, 5, None)
        service: <class 'Dcel.Dcel'>:one=1,two=2
        value: <class 'str'>:1
        _map: <class 'NoneType'>:None
        _dir: <class 'NoneType'>:None
        
address: <class 'str'>:.
        abspath: <class 'str'>:.
        service: <class 'ValueSvc.ValueSvc'>:<ValueSvc.ValueSvc object at 0x7f529251a2c0>
        value: <class 'str'>:
one=1,two=2 three four
five six seven eight 23 4

        _map: <class 'NoneType'>:None
        _dir: <class 'NoneType'>:None
        


In [48]:
# update the fragment
x['1']['spec']['one'].value = '5'

In [49]:
# now the 'sample' text Dcel will have a _map attribute
print(sample.inspect())

address: <class 'str'>:.
        abspath: <class 'str'>:.
        service: <class 'ValueSvc.ValueSvc'>:<ValueSvc.ValueSvc object at 0x7f529251a2c0>
        value: <class 'str'>:
one=5,two=2 three four
five six seven eight 23 4

        _map: <class 'dict'>:{1: (23, <Dcel.Dcel object at 0x7f529251ba30>)}
        _dir: <class 'NoneType'>:None
        


In [32]:
# test for `isdir()`
from DictFS import DictFS
xd = Dcel(x,service_class=DictFS)

In [36]:
# test for `isdir() continued...
xd['1']['spec']['one']['5'].isdir()

True

In [44]:
print(xd['1']['spec']['one']['5'].isdir())

True


In [34]:
print(x['1']['spec'])
print(x['1']['spec']['one'])

{'one': <Dcel.Dcel object at 0x7f529251b2b0>, 'two': <Dcel.Dcel object at 0x7f529251b2e0>}
5


In [9]:
x['1']['spec']['one'].value = "uno"

In [10]:
print(x['1']['spec'])
print(x['1']['spec']['one'])

{'one': <Dcel.Dcel object at 0x7f52ac13d180>, 'two': <Dcel.Dcel object at 0x7f52ac13d150>}
uno


In [11]:
print(sample)


one=uno,two=2 three four
five six seven eight 23 4



In [12]:
from fs import open_fs
from DictFS import DictFS
from fstab_hg import fstab_hg

In [13]:
files = Dcel("demo-files/fs", service_class=open_fs)

In [14]:
sample = files['@']['etc']['fstab']

In [15]:
x = hiena_mp(fstab_hg,sample)
print(type(x))
print(x)

<class 'dict'>
{'1': {'spec': <Dcel.Dcel object at 0x7f52925099c0>, 'file': <Dcel.Dcel object at 0x7f529250b280>, 'vfstype': <Dcel.Dcel object at 0x7f5292509a80>, 'mntopts': <Dcel.Dcel object at 0x7f5292509a20>, 'freq': <Dcel.Dcel object at 0x7f529250b0a0>, 'passno': <Dcel.Dcel object at 0x7f529250ab90>}, '2': {'spec': <Dcel.Dcel object at 0x7f52925087c0>, 'file': <Dcel.Dcel object at 0x7f529250b1c0>, 'vfstype': <Dcel.Dcel object at 0x7f5292509990>, 'mntopts': <Dcel.Dcel object at 0x7f529250b340>, 'freq': <Dcel.Dcel object at 0x7f52925096f0>, 'passno': <Dcel.Dcel object at 0x7f529250b160>}, '3': {'spec': <Dcel.Dcel object at 0x7f5292508fd0>, 'file': <Dcel.Dcel object at 0x7f5292509030>, 'vfstype': <Dcel.Dcel object at 0x7f5292509720>, 'mntopts': <Dcel.Dcel object at 0x7f529250b1f0>, 'freq': <Dcel.Dcel object at 0x7f529250b250>, 'passno': <Dcel.Dcel object at 0x7f5292508760>}, '4': {'spec': <Dcel.Dcel object at 0x7f5292508550>, 'file': <Dcel.Dcel object at 0x7f5292508310>, 'vfstype': <D

In [16]:
backup_content = str(sample)

In [17]:
xd = Dcel(x, service_class=DictFS)

In [18]:
target = xd['2']['file']
print(target)

{CBUSER}/example/


In [19]:
target.value.value = '{CBUSER}/example/'

In [20]:
print(target)

{CBUSER}/example/


In [21]:
print(sample)

# # Cosm / Etc / FSTab

# the .cosm/etc/fstab used by cburn is shared between hosts. The concept of 'localhost' is centric to a generic host model. A file url is relative to the generic host model, whereas a relative file path is relative to the working directory of cloudburner at runtime on each host.

# experimental: include a hostname in the 'file://' url to limit the scope of a filepath to a specific host.

# experimental: proxy the 'file' protocol and allow subdomain syntax to specify shares. The path component is relative to the share.

# idea: make filepaths relative to the fstab's location, ie: for ./.cosm/etc/fstab the relative root is ../../../



# KLUDGE ALLERT: shortid=<hide> works around shortcoming in Fudge star, which requires all entries to have the same fields if selected.
# ie @/etc/fstab/*/mntopts.cskvp/shortid works if <hide> is present
# ie @/etc/fstab/*/mntopts.cskvp/user errors
# goal is to use 'nouser' or 'nogui' flag after Fudge is fixed.


# file://raygan@ray

In [22]:
## Restore from backup
sample.value = backup_content

In [23]:
import json
from DcelJSONEncoder import DcelJSONEncoder

json.dumps(x,cls=DcelJSONEncoder)


'{"1": {"spec": "file://boot.localhost", "file": "{cburnuser}/example/", "vfstype": "cburnfs", "mntopts": "user,shortid=root,idcard=localuser", "freq": "0", "passno": "0"}, "2": {"spec": "file://fs2.cburn.io", "file": "{CBUSER}/example/", "vfstype": "cburnfs", "mntopts": "user,shortid=FishBo,idcard=localuser", "freq": "0", "passno": "0"}, "3": {"spec": "file://fs3.localhost", "file": "{cburnuser}/example/", "vfstype": "cburnfs", "mntopts": "user,shortid=Bucket9,idcard=localuser", "freq": "0", "passno": "0"}, "4": {"spec": "file://fs4.localhost", "file": "/", "vfstype": "cburnfs", "mntopts": "user,shortid=fs4,idcard=localuser", "freq": "0", "passno": "0"}, "5": {"spec": "http://metafs-redis", "file": "no-mount", "vfstype": "cburnfs-meta", "mntopts": "nouser,userid=\'raygan@raygan.com\',userhome=\'raygan-home\',userurl=\'https://raygan-raygan-com.home.laydbug.io\'"}}'

In [24]:
print(sample)

# # Cosm / Etc / FSTab

# the .cosm/etc/fstab used by cburn is shared between hosts. The concept of 'localhost' is centric to a generic host model. A file url is relative to the generic host model, whereas a relative file path is relative to the working directory of cloudburner at runtime on each host.

# experimental: include a hostname in the 'file://' url to limit the scope of a filepath to a specific host.

# experimental: proxy the 'file' protocol and allow subdomain syntax to specify shares. The path component is relative to the share.

# idea: make filepaths relative to the fstab's location, ie: for ./.cosm/etc/fstab the relative root is ../../../



# KLUDGE ALLERT: shortid=<hide> works around shortcoming in Fudge star, which requires all entries to have the same fields if selected.
# ie @/etc/fstab/*/mntopts.cskvp/shortid works if <hide> is present
# ie @/etc/fstab/*/mntopts.cskvp/user errors
# goal is to use 'nouser' or 'nogui' flag after Fudge is fixed.


# file://raygan@ray

In [25]:
type(x['1']['file']['three'])
d = x['1']['file']['three']
e = x['1']['vfstype']['four']
d.value = "surprise"
e.value = "party"
print(f"d: value: {d.value}, address: {d.address}, service: {d.service}")
json.dumps(x,cls=DcelJSONEncoder)

print(sample)
# WARNING: after running this, the internal map becomes out-of-sync.
# The sample will need to be reparsed and will break the bindings
# to whatever key-value-pairs have changed in the sample.

TypeError: expected str, bytes or os.PathLike object, not slice

In [6]:
import re

data = '1234'
p = re.compile('1234')
m = p.match(data)

print(m.end(0))

x = HienaStr(m[0],m)


4


TypeError: str() argument 2 must be str, not re.Match

In [22]:
# playground to practice attaching meta-data to a dict() object

from collections import namedtuple
Frag = namedtuple('Frag',['start','len','frags'])

class parsetree(dict):
    def __init__(self,datadict=dict(),fragmap=dict()):
        self.update(datadict)
        self.cbfrag = fragmap
        
tree = parsetree({"one":"uno"},{"one":(0,3,{"char":(1,1,None)})})

print(tree)
print(tree.cbfrag)
print(tree.cbfrag['one'][2])

{'one': 'uno'}
{'one': (0, 3, {'char': (1, 1, None)})}
{'char': (1, 1, None)}


In [2]:
d = { 'key': 1 }

d['key']

1

In [34]:
from Dcel import Dcel
from DictFS import DictFS

fstab = Dcel(address=fstabg, 
               service_class=DictFS
              )

a = Dcel(formula=hiena_mp, 
         args=[fstab,sample]
        )

if a.value is a.value:
    print('same')
    
a.value['1']['one'] = 'uno'

b = Dcel(address=a, 
         service_class=DictFS
        )



print(b)

for ea in b.listdir():
    try:
        print(ea)
    except:
        pass


SyntaxError: invalid syntax (402375693.py, line 12)

In [25]:
q = Dcel()
r = Dcel()
s = Dcel({'q':q,'r':r},service_class=DictFS)
for ea in s.listdir():
    print(s[ea])

None
None


In [4]:
import re

line = '^.+$'
word = '\w+'
flags = re.M

entry = [[word,flags],[1,2],dict()]
field = [[word,flags]
    ['spec', 
     'file', 
     'vfstype', 
     'mntopts',
     'freq',
     'passno'], 
     {'freq':'[0-9]+', 
      'passno':'[0-9]+'
     }
]

sample = """
one two three four 1 0
five six seven eight 23 4
"""

def parse(g, text):
    lex = g[0][0]
    fl  = g[0][1] 
    labels = g[1]
    sublex = g[2]
    m = re.findall(lex,text,flags)
    w = { k:v for k,v in zip(labels,m)}
    print(w)
    for ea in sublex:
        if ea in w:
            print(re.match(sublex[ea],w[ea],0))

parse(entry, sample)
parse(field, sample)

<>:8: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:8: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
/private/var/mobile/Containers/Data/Application/93EC6E1D-567D-45D0-B634-138E1BDF830E/tmp/ipykernel_721/1818053981.py:8: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  field = [[word,flags]


TypeError: list indices must be integers or slices, not tuple

In [ ]:
for i in 1-10:
    print(i)

In [ ]:
import re

ruleref_re = re.compile("(?P<surface_id>[/@]?)(?P<rule_id>[^{} ]+)(?P<qty>[{][*][}])?")

class HienaParser:
    def __init__(self, 
                 target=None,
                 grammar=None,
                ):
        self.target = target
        self.grammar = grammar
    
    def run_lex(self, 
                text:str,
                regex_args:list,
                ) -> list:
        if not type(regex_args) is list:
            raise TypeError('Requires a list of args suitable for re.findall()')
        _re = regex_args[0]
        try:
            _flags = regex_args[1]
        except:
            _flags = 0
        return re.findall(_re, text, _flags)
        
    def parse_rule_reference(self,ref):
        m = re.match(ruleref_re,ref,0)
        surfaceid = ''
        args = [ ref ]
        return (surfaceid,args)
        
    def run_rule(self, 
                 target:str = None,
                 rulename:str = "",
                 quantity:str = "*",
                ):
        rulepart = self.grammar[rulename]
        print('rulepart: '+rulepart)
        # for first element
        i = 0
        try:
            self.run_lex(rulepart[0])
            i += 1
        except:
            pass  

        def process_rulebody(e):
            try:
                a = self.parse_rule_reference(e)
                res = self.run_rule(*a[1])
            except:
                raise
                
        if type(rulepart) is str:
            process_rulebody(rulepart)
            
        if type(rulepart) is list:
            # loop over rulebody
            for e in rulepart[i:]:
                print(e)
                process_rulebody(e)
        
    def run(self):
        startname = self.grammar["$__start__"]
        return self.run_rule(self.target, 
                             rulename=startname
                          )
                        
def hiena1(target: str, grammar: dict) -> (map, dict):
    mapp = { k: re.findall(grammar[k],
                 target)
            for k in grammar }
    dirr = None
    return (mapp,dirr)

def hiena(target: str, grammar: dict) -> (map, dict):
    parser = HienaParser(target,grammar)
    return parser.run()
    

In [ ]:
import re
surface_id = "(?P<surface_id>[/@]?)"
rule_id = "(?P<rule_id>[^{}+*? ]+)"
qty = "(?P<qty>[+*?]|(?:[{][1-9]+[}]))?"
carver = "([{][^{}]*[}])"
carvers = "(?:\W*"+carver+")+"
_="\W*"+carver
ruleref = surface_id+rule_id+qty+carvers
quantifier = "([*])|([1-9])|([^*, ]+)"
ruleref_re = re.compile(ruleref)
quantifier_re = re.compile(quantifier)

In [ ]:
import re
fstabGrammar = {
    "$__start__": "fstab",
    "fstab": [ "/entry+" ],
    "entry": [ "ENTRY{2}"
               "{@field+:spec,file,vfstype,mntopts,freq:digit,passno:digit} {other} {such}" ], 
    "ENTRY": [[r"^[^#\n]+", re.M]],
    "field": [[r"[^# ]+"]],
    "digit": [[r"[0-9]"]], 
    
    " " : " "
}

In [ ]:
print(fstabGrammar['entry'][0])

In [ ]:
a = re.search( 
    ruleref_re,
    fstabGrammar['entry'][0], 
    0 
)
print(a.groups())

b = re.findall( 
    carver,
    fstabGrammar['entry'][0], 
    0 
)
print(b)

In [ ]:
from fs.osfs import OSFS
from Dcel import Dcel

d = Dcel(address='fs', 
         service_class=OSFS
        )
text = d.path_lookup('.cosm/etc/fstab').value


In [ ]:
s = '/entry{*}'
print(s[0])
print(s[1:].split('{')[1][0])
print()

strings = [ '/entry{*}', 
           '/entry',
           'entry{*}',
           'entry'
          ]

import re
for s in strings:
    c = re.compile("(?P<surface_id>[/@]?)(?P<rule_id>[^{} ]+)(?P<qty>[{][*][}])?")
    m = re.match(c,s,0)
    print(m.groupdict())

In [ ]:
res = hiena(text,fstabGrammar)

In [ ]:
print(res)

In [ ]:
grammar = {"word": r"[^ ]+"}
hiena("one two three", grammar)

In [ ]:
g = {'fs_entry': r"(.+)\n"}
d = """
sftp://example.com  /  sftpfs

localhost:/example  /  file

files.example.com   /  webdavfs
"""
hiena(d,g)